## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
# Импорт необходимых библиотек
import warnings
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
warnings.filterwarnings('ignore')

In [2]:
# Загрузка датасета и изучение его свойств
filepath = '/data.csv'
credit_scoring = pd.read_csv(filepath)
display(credit_scoring.sample(5))
credit_scoring.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16544,1,-2612.575716,42,среднее,1,женат / замужем,0,F,сотрудник,0,125415.385617,покупка жилья
10203,0,-681.569580,34,среднее,1,женат / замужем,0,M,сотрудник,0,272971.868520,операции со своей недвижимостью
9242,1,-1132.854690,39,среднее,1,женат / замужем,0,F,сотрудник,0,153463.745931,покупка коммерческой недвижимости
20352,0,341429.483363,61,среднее,1,женат / замужем,0,F,пенсионер,0,74181.542979,на покупку своего автомобиля
19738,0,-139.170571,62,среднее,1,женат / замужем,0,F,сотрудник,0,233267.420804,автомобиль


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
# Поищем пропущенные значения
print(credit_scoring.isnull().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


### Вывод

Для решения поставленной задачи загружен файл 'data.csv' с данными. Получена таблица из 12 столбцов, 21225 строк. В столбцах days_employed и total_income имеются пропуски. Количество пропусков посчитано методом .isnull().sum(). В столбце days_employed встречаются как отрицательные, так и положительные значения. 

### Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
# Обработка обнаруженных пропусков
# Пропуски в общем трудовом стаже можно заполнить средними значениями
days_employed_mean = credit_scoring['days_employed'].mean()
credit_scoring['days_employed'] = credit_scoring['days_employed'].fillna(
    value=days_employed_mean)

# Пропуски в столбце Ежемесячный доход заполним медианами
median_incomes = credit_scoring.groupby('income_type')['total_income'].median()


def filler_func(row):
    if pd.isna(row['total_income']):
        return median_incomes[row['income_type']]
    return row['total_income']


credit_scoring['total_income'] = credit_scoring.apply(filler_func, axis=1)

credit_scoring.info()
credit_scoring.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
305,2,-7381.422129,36,среднее,1,в разводе,3,F,сотрудник,1,45033.203067,недвижимость
16777,2,-4973.693154,46,Высшее,0,женат / замужем,0,F,сотрудник,0,108804.240185,строительство недвижимости
767,0,367177.825355,59,среднее,1,вдовец / вдова,2,F,пенсионер,0,205155.961397,жилье
16768,0,-3229.254557,45,среднее,1,в разводе,3,F,госслужащий,0,183895.525945,высшее образование
4053,0,-2761.778537,29,ВЫСШЕЕ,0,женат / замужем,0,M,компаньон,0,262944.626017,образование


### Вывод

В полученной таблице обнаружены пропуски в столбцах 'days_employed' и 'total_income'. Эти пропуски имеют следующую зависимость: если в столбце 'days_employed' стоит пропуск, так же пропуск обнаруживается в столбце 'total_income'. Вероятно это случайная ошибка, произошедшая при выгрузке данных. Значения в столбце 'days_employed' никак не влияют на решение поставленной задачи, поэтому были заполнены средним значением по столбцу. Пропуски в столбце 'total_income' заполнены медианными значениями по группам 'income_type', поскольку значение суммарного дохода необходимо знать для верного решения задачи.

### Замена типа данных

In [5]:
# выполним замену методом .astype()
credit_scoring['days_employed'] = credit_scoring['days_employed'].astype('int')
credit_scoring['total_income'] = credit_scoring['total_income'].astype('int')
display(credit_scoring.sample(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6421,0,-428,32,высшее,0,гражданский брак,1,M,сотрудник,0,312283,сделка с подержанным автомобилем
17337,0,333488,67,среднее,1,женат / замужем,0,F,пенсионер,0,455052,покупка своего жилья
4197,0,63046,22,высшее,0,гражданский брак,1,F,компаньон,0,172357,сыграть свадьбу
21066,1,-843,44,среднее,1,женат / замужем,0,M,сотрудник,0,85363,покупка жилья для сдачи
7480,0,-7865,47,Среднее,1,в разводе,3,F,сотрудник,0,72848,сделка с подержанным автомобилем


### Вывод

На данном этапе проведена замена вещественного типа данных на целочисленный в столбцах 'days_employed' и 'total_income'. Для вышеуказанных данных удобнее всего применять метод .astype(), поскольку преобразуем числовые данные в числовые. Метод to_numeric() на данном этапе не применим, поскольку применяется для преобразования данных типа object в тип float64.

### Обработка дубликатов

In [6]:
# Поищем "скрытые" дубликаты
display(credit_scoring['education'].value_counts())

display(credit_scoring['family_status'].value_counts())

display(credit_scoring['income_type'].value_counts())

display(credit_scoring['purpose'].value_counts())

# В столбце 'education' избавимся от них с помощью понижения регистра
credit_scoring['education_lowercase'] = credit_scoring['education'].str.lower()
display(credit_scoring['education_lowercase'].value_counts())

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education_lowercase, dtype: int64

In [7]:
# Поищем явные дубликаты и избавимся от них
display(credit_scoring.duplicated().sum())
credit_scoring = credit_scoring.drop_duplicates().reset_index(drop=True)

credit_scoring.info()

54

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   children             21471 non-null  int64 
 1   days_employed        21471 non-null  int32 
 2   dob_years            21471 non-null  int64 
 3   education            21471 non-null  object
 4   education_id         21471 non-null  int64 
 5   family_status        21471 non-null  object
 6   family_status_id     21471 non-null  int64 
 7   gender               21471 non-null  object
 8   income_type          21471 non-null  object
 9   debt                 21471 non-null  int64 
 10  total_income         21471 non-null  int32 
 11  purpose              21471 non-null  object
 12  education_lowercase  21471 non-null  object
dtypes: int32(2), int64(5), object(6)
memory usage: 2.0+ MB


### Вывод

Для поиска дубликатов был использован метод value_counts(), возвращающий уникальные значения с подсчетом частоты их появления. Применение метода duplicated не представляется возможным, так как изучать таблицу с булевыми значениями на 21525 строк совершенно не рационально. В сочетании с методом sum() метод неинформативен.
Ручной поиск дубликатов в столбцах 'education', 'family_status', 'income_type', 'purpose' дал следующие результаты:
1) в столбцах 'family_status', 'income_type' дубликаты отсутствуют;
2) в столбце 'education' от дубликатов можно избавиться снижением регистра;
3) в столбце 'purpose' обнаружено значительное количество дубликатов. Избавиться от них возможно методом лемматизации 

Выполнен поиск дубликатов и применен метод drop_duplicates().sum()

### Лемматизация

In [8]:
# Создадим список со значениями столбца purpose для ускорения лемматизации
purpose_list = credit_scoring['purpose'].unique()
display(purpose_list)

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [9]:
# Выполним лемматизацию
lemmas = []

for i in range(38):
    lemmas.append(m.lemmatize(purpose_list[i]))

display(lemmas)

[['покупка', ' ', 'жилье', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['дополнительный', ' ', 'образование', '\n'],
 ['сыграть', ' ', 'свадьба', '\n'],
 ['операция', ' ', 'с', ' ', 'жилье', '\n'],
 ['образование', '\n'],
 ['на', ' ', 'проведение', ' ', 'свадьба', '\n'],
 ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'],
 ['покупка', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'],
 ['недвижимость', '\n'],
 ['строительство', ' ', 'недвижимость', '\n'],
 ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'],
 ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'],
 ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['жилье', '\n'],
 ['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n'],
 ['автомобиль'

In [10]:
# Создадим словарь по результатам проведенной лемматизации
lemm_dict = dict(zip(purpose_list, lemmas))

In [11]:
# Заменим значения столбца purpose на лемматизированные
credit_scoring['purpose'] = credit_scoring['purpose'].map(lemm_dict)
display(credit_scoring.sample(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase
11357,2,-228,29,среднее,1,женат / замужем,0,F,компаньон,0,130334,"[получение, , высокий, , образование, \n]",среднее
18051,1,-1822,27,среднее,1,гражданский брак,1,F,сотрудник,0,107884,"[сыграть, , свадьба, \n]",среднее
12815,0,-1702,25,среднее,1,женат / замужем,0,M,компаньон,0,168348,"[образование, \n]",среднее
7354,0,354169,56,среднее,1,женат / замужем,0,F,пенсионер,0,62218,"[получение, , дополнительный, , образование,...",среднее
18520,0,63046,54,ВЫСШЕЕ,0,женат / замужем,0,M,пенсионер,0,118514,"[заниматься, , образование, \n]",высшее


In [12]:
# Создадим функцию для создания столбца upgraded_purpose
# с лемматизированными значениями целей кредита

def updating_purpose(row):
    if 'свадьба' in row['purpose']:
        return 'свадьба'
    elif 'автомобиль' in row['purpose']:
        return 'автомобиль'
    elif 'недвижимость' in row['purpose']:
        return 'недвижимость'
    elif 'жилье' in row['purpose']:
        return 'недвижимость'
    elif 'образование' in row['purpose']:
        return 'образование'


credit_scoring['updated_purpose'] = credit_scoring.apply(
    updating_purpose, axis=1)
credit_scoring.info()

display(credit_scoring['updated_purpose'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   children             21471 non-null  int64 
 1   days_employed        21471 non-null  int32 
 2   dob_years            21471 non-null  int64 
 3   education            21471 non-null  object
 4   education_id         21471 non-null  int64 
 5   family_status        21471 non-null  object
 6   family_status_id     21471 non-null  int64 
 7   gender               21471 non-null  object
 8   income_type          21471 non-null  object
 9   debt                 21471 non-null  int64 
 10  total_income         21471 non-null  int32 
 11  purpose              21471 non-null  object
 12  education_lowercase  21471 non-null  object
 13  updated_purpose      21471 non-null  object
dtypes: int32(2), int64(5), object(7)
memory usage: 2.1+ MB


недвижимость    10814
автомобиль       4308
образование      4014
свадьба          2335
Name: updated_purpose, dtype: int64

In [13]:
# Последним шагом удалим столбец с леммами целей кредита
credit_scoring = credit_scoring.drop('purpose', axis=1)

### Вывод

Проведена лемматизация данных столбца 'purpose'. Результатом получен столбец с обновленными данными по целям получения кредитов - 'updated_purpose'. Комментарий: леммы 'недвижимость' и 'жилье' объединены в общее наименование 'недвижимость' для цели кредита. Леммы 'автомобиль','свадьба' и 'образование' были взяты без изменений. Методом value_counts() посчитаны уникальные данные полученного столбца. 

### Категоризация данных

In [14]:
# Проведем категоризацию данных
credit_scoring_query = credit_scoring[[
    'children', 'family_status', 'updated_purpose', 'total_income', 'debt']]

credit_scoring_query.loc[credit_scoring_query['children'] < 0, 'children'] = 1
credit_scoring_query.loc[credit_scoring_query['children']
                         == 20, 'children'] = 2
credit_scoring_query = credit_scoring_query.drop_duplicates().reset_index(drop=True)

print('--------------------------------------------------------------------------------------------------------')


def children_status(count1):
    if count1 < 1:
        return 'без детей'
    elif count1 >= 3:
        return 'многодетные'
    return 'с детьми'


credit_scoring_query['children_status'] = credit_scoring_query['children'].apply(
    children_status)


def income_status(count2):
    if count2 < credit_scoring['total_income'].quantile(q=0.25):
        return 'низкий доход'
    if count2 >= credit_scoring['total_income'].quantile(q=0.75):
        return 'высокий доход'
    return 'средний доход'


credit_scoring_query['income_status'] = credit_scoring_query['total_income'].apply(
    income_status)


def marriage_status(row):
    if 'женат / замужем' in row['family_status']:
        return 'в браке'
    elif 'гражданский брак' in row['family_status']:
        return 'в браке'
    elif 'вдовец / вдова' in row['family_status']:
        return 'не в браке'
    elif 'в разводе' in row['family_status']:
        return 'не в браке'
    return 'не в браке'


credit_scoring_query['marriage_status'] = credit_scoring_query.apply(
    marriage_status, axis=1)


def purpose_status(row):
    if 'образование' in row['updated_purpose']:
        return 'первостепенная цель'
    elif 'жилье' in row['updated_purpose']:
        return 'первостепенная цель'
    elif 'недвижимость' in row['updated_purpose']:
        return 'первостепенная цель'
    return 'второстепенная цель'


credit_scoring_query['purpose_status'] = credit_scoring_query.apply(
    purpose_status, axis=1)

credit_scoring_query.info()
print('----------------------------------------------------------')
display(credit_scoring_query.head())

--------------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19525 entries, 0 to 19524
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   children         19525 non-null  int64 
 1   family_status    19525 non-null  object
 2   updated_purpose  19525 non-null  object
 3   total_income     19525 non-null  int32 
 4   debt             19525 non-null  int64 
 5   children_status  19525 non-null  object
 6   income_status    19525 non-null  object
 7   marriage_status  19525 non-null  object
 8   purpose_status   19525 non-null  object
dtypes: int32(1), int64(2), object(6)
memory usage: 1.3+ MB
----------------------------------------------------------


,children,family_status,updated_purpose,total_income,debt,children_status,income_status,marriage_status,purpose_status
0,1,женат / замужем,недвижимость,253875,0,с детьми,высокий доход,в браке,первостепенная цель
1,1,женат / замужем,автомобиль,112080,0,с детьми,средний доход,в браке,второстепенная цель
2,0,женат / замужем,недвижимость,145885,0,без детей,средний доход,в браке,первостепенная цель
3,3,женат / замужем,образование,267628,0,многодетные,высокий доход,в браке,первостепенная цель
4,0,гражданский брак,свадьба,158616,0,без детей,средний доход,в браке,второстепенная цель


### Вывод

На этапе выполнения этого задания в столбце 'children' были обнаружены артефакты '-1' и '20' и заменены на значения "1" и "2" соответственно. Данные ошибки тоже можно отнести к категории случайных при выгрузке исходных данных. Комментарий: семей с 20 детьми в мире очень мало, а пользователей кредитными продуктами банков еще меньше, поэтому число '20' заменено на '2'. Проведена категоризация по столбцам: 'children' - выбраны подкатегории 'без детей', 'с детьми' - 1-2 ребенка на семью и 'многодетные' - 3 ребенка и более на семью; 'total_income' - выбраны подкатегории 'низкий доход' с доходом менее 0.25 квантиля, 'высокий доход' - с доходом выше 0.75 квантиля, 'средний доход' - все остальные; 'family_status' - 'в браке' - для пар находящихся в официальном и гражданском браках, 'не в браке' - для всех остальных пар. Категоризация столбца 'updated_purpose' на подкатегории 'первостепенная цель' и 'второстепенная цель' при ответе на вопрос о зависимости между целью кредита и возвратом кредита в срок не даёт уточненной информации и в итоге просто использовалась для подсчета строк в сшитых таблицах.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [15]:
children_debt1 = credit_scoring_query.pivot_table(
    index=['children_status'], columns='debt', values='children', aggfunc='count')

children_debt1['reliability'] = children_debt1[0] / \
    (children_debt1[1] + children_debt1[0])
display(children_debt1.sort_values(by='reliability', ascending=False))
print('-----------------------------------------------------')
children_debt2 = credit_scoring_query.pivot_table(
    index=['children_status', 'children'], columns='debt', values='marriage_status', aggfunc='count')
children_debt2['reliability'] = children_debt2[0] / \
    (children_debt2[1] + children_debt2[0])
display(children_debt2.sort_values(by='reliability', ascending=False))

debt,0,1,reliability
children_status,,,
без детей,11772,986,0.922715
многодетные,329,30,0.916435
с детьми,5784,624,0.902622


-----------------------------------------------------


,debt,0,1,reliability
children_status,children,,,
без детей,0,11772.0,986.0,0.922715
многодетные,3,284.0,26.0,0.916129
с детьми,1,4020.0,428.0,0.903777
многодетные,4,36.0,4.0,0.900000
с детьми,2,1764.0,196.0,0.900000
многодетные,5,9.0,NaN,NaN


### Вывод

Зависимость между наличием детей и сроком возврата кредита есть. В конечную таблицу добавлен условный параметр 'reliability' , который отражает надежность той или иной группы заемщиков. Семьи без детей являются наиболее надежными заемщиками - параметр надежности на уровне 92%, а семьи с детьми являются самыми ненадежными - показатель надежности на уровне 90 %. Для более точной информации о заемщиках с детьми создана еще одна таблица. По ней видно, что самыми надежными замщиками являются многодетные семьи с 3 детьми. Примерно на одном уровне находятся семьи с одним, двумя, четырьмя детьми. Для семей с 5 детьми и более получено значение NaN, однако этим значением можно пренебречь, поскольку таких семей всего 9 - менее 0,5 % от общего числа заемщиков. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [16]:
family_debt1 = credit_scoring_query.pivot_table(
    index=['marriage_status'], columns='debt', values='family_status', aggfunc='count')

family_debt1['reliability'] = family_debt1[0] / \
    (family_debt1[0] + family_debt1[1])
display(family_debt1.sort_values(by='reliability', ascending=False))
print('--------------------------------------------------------------')
family_debt2 = credit_scoring_query.pivot_table(
    index=['marriage_status', 'family_status'], columns='debt', values='children_status', aggfunc='count')
family_debt2['reliability'] = family_debt2[0] / \
    (family_debt2[0] + family_debt2[1])
display(family_debt2.sort_values(by='reliability', ascending=False))

debt,0,1,reliability
marriage_status,,,
в браке,13735,1232,0.917686
не в браке,4150,408,0.910487


--------------------------------------------------------------


debt                                       0    1  reliability
marriage_status family_status                                 
не в браке      вдовец / вдова           821   61     0.930839
                в разводе               1031   83     0.925494
в браке         женат / замужем        10297  872     0.921927
                гражданский брак        3438  360     0.905213
не в браке      Не женат / не замужем   2298  264     0.896956

### Вывод

Зависимость между семейным положением и сроком возврата кредита есть. В конечную таблицу добавлен условный параметр 'reliability' , который отражает надежность той или иной группы заемщиков. Надежность пар, состоящих в браке, и пар, состоящих не в браке, находится на одном уровне - 91-92 %. Для более точной информации о заемщиках создана еще одна таблица, посмотрев на которую можно сделать вывод о том, что вдовы/вдовцы являются самыми надежными заемщиками. Соотношение надежности для женатых/замужних и разведенных сопоставимо - 92 %. Среди одиноких заемщиков и пар, живущих в гражданском браке показатель надежности самый низкий - 89 % и 90 % соответственно. В целом, здесь видно, что наиболее надежные заемщики, это те, кто находится или когда-либо был в официальном браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [17]:
income_debt = credit_scoring_query.pivot_table(
    index=['income_status'], columns='debt', values='total_income', aggfunc='count')

income_debt['reliability'] = income_debt[0]/(income_debt[0] + income_debt[1])
display(income_debt.sort_values(by='reliability', ascending=False))

debt,0,1,reliability
income_status,,,
высокий доход,4981,383,0.928598
низкий доход,4922,427,0.920172
средний доход,7982,830,0.905810


### Вывод

Зависимость между уровнем дохода и сроком возврата кредита есть. В конечную таблицу добавлен условный параметр 'reliability' , который отражает надежность той или иной группы заемщиков. Лидируют по надежности заемщики с высоким доходом - примерно 93 % от общего числа заемщиков. Уровень надежности заемщиков со низким доходом сопоставим  - 92 %. Самый низкий показатель надежности у заемщиков с средним уровнем дохода - 90 %.

- Как разные цели кредита влияют на его возврат в срок?

In [18]:
purpose_debt = credit_scoring_query.pivot_table(
    index=['updated_purpose'], columns='debt', values='purpose_status', aggfunc='count')

purpose_debt['reliability'] = purpose_debt[0] / \
    (purpose_debt[0] + purpose_debt[1])
display(purpose_debt.sort_values(by='reliability', ascending=False))

debt,0,1,reliability
updated_purpose,,,
недвижимость,9063,740,0.924513
свадьба,1951,165,0.922023
образование,3307,349,0.904540
автомобиль,3564,386,0.902278


### Вывод

По испольуемому в предыдущих задачах условному показателю можно предположить, что к операциям с недвижимостью и организации свадеб заемщики относятся более ответственно - уровень надежности 92 %. Менее надежны обучающиеся и автолюбители - уровень надежности равен 90 %.

### Шаг 4. Общий вывод

Проведено исследование надежности заемщиков по заказу кредитного отдела банка. Сделан ряд выводов: 
- 1) существует обратно пропорциональная связь между наличием детей и сроком возврата кредита (чем меньше детей в семье, тем выше надежность заемщика); 
- 2) существует связь между семейным положением заемщика и сроком возврата кредита;
- 3) существует прямая связь между суммарным доходом заемщика и сроком возврата кредита;
- 4) цель кредита также влияет на решение банка о выдаче кредита.
В результате, можно сделать вывод, что на все четыре вопроса задачи найдены ответы. 

Рекомендации для банка:
    
- а) лучше выдавать кредиты либо бездетным клиентам, либо клиенты с 3 детьми, менее привлекательные клиенты для банка - семьи с 1, 2, 4 детьми;
- б) лучше выдавать кредиты женатым парам, разведенным парам, вдовцам/вдовам, одинокие заемщики и пары, живущие гражданским браком - менее привлекательны для кредитования;
- в) что касается доходов, безусловно клиенты с высоким доходом лучшие заемщики, клиенты с низкими доходами тоже не отстают, а вот клиенты со средним доходом менее привлекательны для банка. (Комментарий: мне кажется, что это связано с тем, что: клиенту с высоким доходом кредит выплатить проще; клиент с низким доходом банку выгоден из-за того, что возможны просрочки, а с ними и выплаты пени и т.п; а вот клиенты со средним доходом - возможно случайность, либо проблемы в бизнесе);
- г) в зависимости от целей банку лучше выдавать кредиты на недвижимость и организацию свадеб (ведь ипотечная квартира находится в собственности банка - это своебразная страховка от невыплаты, а женатая пара - более привлекательна для банка - это было выяснено в пункте 4.2), на учебу и покупку/ремонт автомобиля кредиты выдавать с осторожностью.
